## **Machine Learning - WNBA Playoffs Prediction**
This notebook will focus on the undestanding of the data. We will be using SQLite to store the data due to its scalability & the fact that it's a relational schema.

https://docs.python.org/3/library/sqlite3.html

Import sqlite3 and connect to database file

### **Imports**

In [1]:
import pandas as pd
import sqlite3
import prep_utils as pu 
import sys
import os
import seaborn as sns
import matplotlib.pyplot as plt

### **Database Connection Setup**

In [2]:
db = sqlite3.connect("db/ac.db")
db_cur = db.cursor()

[df_awards, df_coaches, df_players_teams, df_players, df_series_post, df_teams_post, df_teams] = pu.db_to_pandas(db)

***Prepare Coaches Dataframe***

In [3]:
df_new_coaches = pu.prepare_coaches(df_coaches, df_awards,10)
pu.group_coaches(df_new_coaches)

Dropping Attribute lgID in Coaches...
Creating attribute coach previous regular season win ratio...
Creating attribute coach playoffs win ratio...
Creating attribute coach playoffs count...
Creating attribute coach awards count...
Dropping attribute post_wins..
Dropping attribute post_losses..
Dropping attribute won..
Dropping attribute lost..

Coaches Null Verification:
year                    0
tmID                    0
coachID                 0
coach_reg_season_wr     0
coach_po_season_wr      0
coach_playoffs_count    0
coach_awards            0
dtype: int64


,year,tmID,coachID,coach_reg_season_wr,coach_po_season_wr,coach_playoffs_count,coach_awards
0,1,CHA,dunntr01wc,0.000000,0.000000,0,0
1,1,CLE,hugheda99w,0.000000,0.000000,0,0
2,1,DET,liebena01w,0.000000,0.000000,0,0
3,1,HOU,chancva99w,0.000000,0.000000,0,0
4,1,IND,donovan99w,0.000000,0.000000,0,0
...,...,...,...,...,...,...,...
137,10,PHO,gaineco01w,0.470588,0.000000,0,0
138,10,SAC,bouceje01w,0.544118,0.333333,2,0
139,10,SAS,hugheda99w,0.488722,0.423077,5,2
140,10,SEA,aglerbr99w,0.470085,0.333333,1,0


***Prepare Players Dataframe***


In [4]:
df_new_players_teams = pu.prepare_player_teams(df_players_teams,df_awards,10)

Dropping Attribute lgID in Players_Teams...
        playerID  year tmID         GP         GS      minutes      points   oRebounds   dRebounds    rebounds     assists     steals      blocks   turnovers          PF  fgAttempted      fgMade  ftAttempted      ftMade  threeAttempted  threeMade        dq     PostGP    PostGS  PostMinutes  PostPoints  PostoRebounds  PostdRebounds  PostRebounds  PostAssists  PostSteals  PostBlocks  PostTurnovers     PostPF  PostfgAttempted  PostfgMade  PostftAttempted  PostftMade  PostthreeAttempted  PostthreeMade    PostDQ  player_awards
0     abrossv01w     2  MIN   0.000000   0.000000     0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   0.000000    0.000000    0.000000    0.000000     0.000000    0.000000     0.000000    0.000000        0.000000   0.000000  0.000000   0.000000  0.000000     0.000000    0.000000       0.000000       0.000000      0.000000     0.000000    0.000000    0.000000       0.000000   0.000000         0.000000  

***Prepare Teams Dataframe***

In [5]:
new_teams = pu.prepare_teams(df_teams,df_teams_post,3)
print(new_teams.to_string())

Dropping divID in Teams...
Dropping ldID in Teams...
Dropping seeded in Teams...
Dropping tmORB, tmDRB, tmTRB, opptmORB, opptmDRB, opptmTRB in Teams...
Dropping GP, homeW, homeL, awayW, awayL, confW, confL, attend, name, confID, franchID & arena in Teams...
Converting Target PLAYOFF to binary onTeams...
Creating attribute winrate Teams...
Dropping won & lost in Teams...
Creating attribute PlayOffs winrate Teams...
     year tmID      rank  playoff        o_fgm        o_fga       o_ftm       o_fta       o_3pm       o_3pa      o_oreb      o_dreb        o_reb      o_asts        o_pf       o_stl        o_to       o_blk        o_pts        d_fgm        d_fga       d_ftm       d_fta       d_3pm       d_3pa      d_oreb      d_dreb        d_reb      d_asts        d_pf       d_stl        d_to       d_blk        d_pts   min   Winrate  PO_Winrate
0       9  ATL  0.000000        0     0.000000     0.000000    0.000000    0.000000    0.000000    0.000000    0.000000    0.000000     0.000000    0.00

In [6]:
df_new_player_rankings = pu.prepare_players_for_ranking(df_players_teams, df_awards)
feature_importance, df_new_players = pu.feature_importance_players(df_new_player_rankings, df_players,df_teams)



Mean Squared Error for G: 0.2556188976377953
Feature importance for G:
fg%: 0.13193237302924718
PER: 0.10044591515069988
3pt%: 0.08983233557532318
PPM: 0.08768905971755102
PF: 0.0840080924402953
ft%: 0.08248050232600053
assists: 0.08115520424238572
turnovers: 0.07321004802945645
rebounds: 0.05201427474303845
steals: 0.05181883354033162
dRebounds: 0.05160571503573023
oRebounds: 0.05155491171357774
blocks: 0.04951945819906815
dq: 0.011413541390102831
player_awards: 0.0013197348671915427

Mean Squared Error for C-F: 0.3313444444444444
Feature importance for C-F:
blocks: 0.23423574689189325
assists: 0.11549505063910384
PER: 0.09043045077136809
turnovers: 0.08887239092119022
fg%: 0.0749682340526368
ft%: 0.07346679080161606
PPM: 0.07253536518044842
oRebounds: 0.058840082837705705
dRebounds: 0.04848657108330218
PF: 0.04098213683532461
steals: 0.038571410368978154
3pt%: 0.03256304128415902
rebounds: 0.02416353419307966
dq: 0.006389194139194141
player_awards: 0.0

Mean Squared Error for C: 0.2

In [7]:
rank_players_regular = pu.ranking_players(feature_importance, df_new_players)
print('Best players in the regular season: ')
print(rank_players_regular)


Best players in the regular season: 
        playerID  year    rating
1074  jacksla01w     8  0.505304
1655  leslili01w     5  0.497612
1155  parkeca01w     9  0.495831
204   jacksla01w     4  0.495122
1580  catchta01w     3  0.492006
...          ...   ...       ...
959   oneilkr01w     9  0.044325
1372  gaithka01w     3  0.044077
543   berezva01w     9  0.043292
860   weberma01w     8  0.043214
1531  chambco01w     8  0.000000

[1805 rows x 3 columns]


In [8]:
rank_playoff_players = pu.ranking_playoff_players(feature_importance, df_new_players)
print('Best players in the playoffs: ')
print(rank_playoff_players)

Best players in the playoffs: 
        playerID  year  PostRating
395    zollsh01w     9    0.062300
1393  zirkozu01w     4    0.062300
657   zellosh01w    10    0.359037
835    zarafr01w     6    0.358208
1322  zakalok01w     1    0.051277
...          ...   ...         ...
33    abrossv01w     5    0.226451
45    abrossv01w     6    0.069610
59    abrossv01w     7    0.069610
81    abrossv01w     9    0.233608
0     abrossv01w     2    0.069610

[1805 rows x 3 columns]


In [9]:
power_ratings = pu.team_power_rating(df_teams, df_new_players)


sorted_power_ratings = power_ratings.sort_values(by=['year', 'PowerRating'], ascending=[True, False])
print(sorted_power_ratings)
# Group by year and select the top 6 teams
top_teams_by_year = sorted_power_ratings.groupby('year').head(8)

# Count how many of the top 6 teams for each year made the playoffs
playoffs_made_by_year = top_teams_by_year.groupby('year')['playoff'].apply(lambda x: (x == 'Y').sum()).reset_index()

# Print or use the results
for index, row in playoffs_made_by_year.iterrows():
    print('Year ' + str(row['year']) + ' based on Power Ratings ' + str(row['playoff']) + '/8 best teams made the playoffs')

print('Ranking System Accuracy: ' + str(playoffs_made_by_year['playoff'].sum()/ (8*len(playoffs_made_by_year))) + '%')

     year tmID  PowerRating playoff  rank
5       1  LAS     0.343036       Y     1
8       1  NYL     0.342154       Y     1
3       1  HOU     0.337359       Y     2
12      1  SAC     0.319554       Y     3
9       1  ORL     0.315229       Y     3
..    ...  ...          ...     ...   ...
135    10  MIN     0.275221       N     5
136    10  NYL     0.273368       N     7
131    10  CON     0.266541       N     6
130    10  CHI     0.261638       N     5
138    10  SAC     0.235489       N     6

[142 rows x 5 columns]
Year 1 based on Power Ratings 8/8 best teams made the playoffs
Year 2 based on Power Ratings 8/8 best teams made the playoffs
Year 3 based on Power Ratings 8/8 best teams made the playoffs
Year 4 based on Power Ratings 7/8 best teams made the playoffs
Year 5 based on Power Ratings 7/8 best teams made the playoffs
Year 6 based on Power Ratings 7/8 best teams made the playoffs
Year 7 based on Power Ratings 7/8 best teams made the playoffs
Year 8 based on Power Ratings 8

In [10]:
best_colleges = pu.best_colleges(df_players_teams,df_teams,df_players)


print(best_colleges)

                    college  TotalPlayoffAppearances  CollegeRank
88                Tennessee                       21            1
17              Connecticut                       17            2
31                  Georgia                       15            3
86                 Stanford                       12            4
48           Louisiana Tech                       11            5
..                      ...                      ...          ...
80               Seton Hall                        1           14
25    Florida International                        1           14
82     Southern Mississippi                        1           14
50                    Maine                        1           14
0   Academy of Sport Moscow                        1           14

[113 rows x 3 columns]


In [11]:
awards = pu.player_awards(df_new_players,df_awards)


# get a player, order by year
player = awards[awards['playerID'] == 'leslili01w']
player = player.sort_values(by=['year'], ascending=[True])

print(player)


        playerID  year  award  cumulative_awards
2750  leslili01w     1      0                0.0
2751  leslili01w     2      3                0.0
2752  leslili01w     3      2                3.0
2753  leslili01w     4      0                5.0
2754  leslili01w     5      2                5.0
2755  leslili01w     6      0                7.0
2756  leslili01w     7      2                7.0
2757  leslili01w     8      0                9.0
2758  leslili01w     9      1                9.0
2759  leslili01w    10      0               10.0


In [12]:
teams = pu.team_ratings(sorted_power_ratings)


team = teams[teams['tmID'] == 'HOU']
print(team)

     year tmID  PowerRating playoff  rank  cum_Rating
3       1  HOU     0.337359       Y     2    0.000000
19      2  HOU     0.293389       Y     4    0.337359
35      3  HOU     0.289042       Y     2    0.315374
52      4  HOU     0.310742       Y     2    0.306597
65      5  HOU     0.257664       N     6    0.307633
78      6  HOU     0.277175       Y     3    0.297639
92      7  HOU     0.283724       Y     3    0.294229
105     8  HOU     0.272868       N     5    0.292728
119     9  HOU     0.259133       N     5    0.290245


In [13]:
colleges = pu.teams_colleges(df_new_players,best_colleges,df_teams)

colleges = colleges[colleges['tmID'] == 'IND']

ordered_colleges = colleges.sort_values(by=['year', 'CollegeRank'], ascending=[True, True])

print(ordered_colleges)

   tmID  year  CollegeRank   min  rank
43  IND     1     5.168872  6425     7
44  IND     2     5.448031  6475     6
45  IND     3     4.569650  6425     4
46  IND     4     5.996073  6875     5
47  IND     5     6.759708  6850     6
48  IND     6     4.771264  6925     2
49  IND     7     7.064672  6850     3
50  IND     8     8.205236  6875     2
51  IND     9     6.474676  6950     4
52  IND    10     6.009819  6925     1


Final Table for Testing

In [14]:
df_testing = df_teams

### **Data Preparation**
We will preparate the data in each table, by cleaning & formatting it so that it can be easily used by the machine learning models afterwards.